In [34]:
import numpy as np

In [58]:
batch_size = 10
num_concepts = 10
num_latent_concepts = 5

In [69]:
# sampling from independent normals using parameterization trick
def sample(num_concepts, num_latent_concepts, batch_size):
    mu_hat = np.random.random(num_concepts).reshape(num_concepts, 1) # (n_concepts,  1)
    true_sigma_hat = np.random.random(num_concepts).reshape(num_concepts, 1) # (n_concepts,  1)
    c_hat = np.random.standard_normal(batch_size*num_concepts).reshape(batch_size, num_concepts, 1) * true_sigma_hat**0.5 + mu_hat # (batch_size, n_concepts,  1)

    mu_tilde = np.random.random(num_latent_concepts).reshape(num_latent_concepts, 1) # (n_latent,  1)
    true_sigma_tilde = np.random.random(num_latent_concepts).reshape(num_latent_concepts, 1) # (n_latent,  1)
    c_tilde = np.random.standard_normal(batch_size*num_latent_concepts).reshape(batch_size, num_latent_concepts, 1) * true_sigma_tilde**0.5 + mu_tilde # (batch_size, n_latent,  1)
    print(list(true_sigma_hat.squeeze())+list(true_sigma_tilde.squeeze()))
    return c_hat, c_tilde

# c_hat_mean = np.mean(c_hat, axis=0) # (n_concepts, 1)
# c_tilde_mean = np.mean(c_tilde, axis=0) # (n_latent, 1)
# print(c_hat.shape, c_tilde.shape)

In [70]:
def loss_term(c_hat, c_tilde):
    c_hat_mean = np.mean(c_hat, axis=0) # (n_concepts, 1)
    c_tilde_mean = np.mean(c_tilde, axis=0) # (n_latent, 1)
    c_tilde_deviation = c_tilde-c_tilde_mean # (batch_size, n_latent, 1)
    c_tilde_deviation_T = np.moveaxis(c_tilde_deviation,-1,-2) # (batch_size, 1, n_latent)
    sigma_tilde = 1/batch_size * np.sum(np.matmul(c_tilde_deviation, c_tilde_deviation_T),axis=0)  # (n_latent, n_latent)
    log_det_sigma_tilde = np.log(np.linalg.det(sigma_tilde)) # scalar

    c_stack_T = np.concatenate((c_hat,c_tilde), axis=1) # (batch_size, n_concepts+n_latent, 1)
    c_hat_mean_resize = np.repeat(c_hat_mean.reshape(1,num_concepts,1), batch_size, axis=0) # (batch_size, n_concepts, 1)
    c_tilde_mean_resize = np.repeat(c_tilde_mean.reshape(1,num_latent_concepts,1), batch_size, axis=0) # (batch_size, n_latent, 1)
    c_stack_mean_T = np.concatenate((c_hat_mean_resize,c_tilde_mean_resize), axis=1)  # (batch_size, n_concepts+n_latent, 1)
    c_stack_deviation = c_stack_T-c_stack_mean_T  # (batch_size, n_concepts+n_latent, 1)
    c_stack_deviation_T = np.moveaxis(c_stack_deviation,-1,-2)  # (batch_size, 1, n_concepts+n_latent)
    sigma = 1/batch_size * np.sum(np.matmul(c_stack_deviation,c_stack_deviation_T),axis=0)  # (batch_size, n_concepts+n_latent, n_concepts+n_latent)
    log_det_sigma = np.log(np.linalg.det(sigma)) # scalar
    print(np.diag(sigma))

    r = (log_det_sigma - log_det_sigma_tilde) / 2
    return r

In [71]:
for _ in range(100):
    c_hat, c_tilde = sample(num_concepts, num_latent_concepts, batch_size)
    r = loss_term(c_hat, c_tilde)
    print(r)

[0.43504073249942077, 0.0053429486053957476, 0.29527597164421326, 0.16210901425737811, 0.45566149079988083, 0.129747473849249, 0.6897697832646151, 0.8160032318653344, 0.673115678977422, 0.9754142043064248, 0.8546337852073266, 0.8235294418749666, 0.8115996069904081, 0.6638363371881754, 0.6828091501931518]
[0.2188211  0.0028143  0.3842356  0.1316441  0.68698881 0.06694651 0.28875514 0.54368211 0.53387738 0.72724419 0.97834212 0.49414475 0.46378446 0.39992021 0.83654699]
nan
[0.5951453523675396, 0.8741918314945482, 0.8547794982840758, 0.46703616450955765, 0.9289851274491387, 0.20818590360036549, 0.35744271908770464, 0.11877306492238071, 0.12561623245270293, 0.4050005039984993, 0.8432030129721331, 0.6495349095131607, 0.5654134901696792, 0.11495127976564223, 0.07791995311580335]
[0.50078479 0.51899276 2.03039212 0.58449589 1.09965311 0.05791676 0.39086339 0.11512001 0.08885088 0.71155188 0.66669265 0.32438712 0.19320568 0.07740839 0.11791557]
nan
[0.00908030751550426, 0.04266062597558817, 0

/tmp/ipykernel_1004/664785942.py:16: RuntimeWarning: invalid value encountered in log
  log_det_sigma = np.log(np.linalg.det(sigma)) # scalar
/tmp/ipykernel_1004/664785942.py:16: RuntimeWarning: divide by zero encountered in log
  log_det_sigma = np.log(np.linalg.det(sigma)) # scalar


In [115]:
print(r)

0.5875920078389916
